# Data exploration of Climate Risk Viewer

In [1]:
#libraries

import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import pyarrow

# To do

## Select a few layers we want to work with census

## identify values that we can transform

## Find features that will help us merge the data

## Calculate an index based on weight of each value that is overlaped

## Map the index 

In [2]:
# downloading files from the CRV


58500

In [2]:
#trying to identify counties by latitude/longitude coordinates using TIGER/Line Shapefile, 2019, nation, U.S., Current County and Equivalent National Shapefile from data.gov

# Load the shapefile
gdf = gpd.read_file('tl_2019_us_county/tl_2019_us_county.shp') #file is too large to commit; download from https://catalog.data.gov/dataset/tiger-line-shapefile-2019-nation-u-s-current-county-and-equivalent-national-shapefile

gdf.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,NaN,NaN,NaN,A,1477652222,10690952,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,NaN,NaN,NaN,A,680962890,61582307,+46.2946377,-123.4244583,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,NaN,NaN,NaN,A,6016819475,29089486,+34.3592729,-104.3686961,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,NaN,A,2169270569,22849484,+40.7835474,-096.6886584,"POLYGON ((-96.91075 40.78494, -96.91075 40.790..."
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,NaN,NaN,NaN,A,1489645188,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."


In [3]:
gdf.shape

(3233, 18)

In [4]:
gdf_CA = gdf[gdf['STATEFP']=='06']

In [5]:
gdf_CA.shape

(58, 18)

In [6]:
gdf_CA.to_csv('gdf_CA.csv')

In [7]:
gdf_CA.to_parquet('gdf_CA.parquet.gzip',
              compression='gzip')  
pd.read_parquet('gdf_CA.parquet.gzip')  

In [10]:
gdf_CA2 = pd.read_parquet('gdf_CA.parquet.gzip')  
gdf_CA2.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
8,06,091,00277310,06091,Sierra,Sierra County,06,H1,G4020,None,None,None,A,2468694583,23299110,+39.5769252,-120.5219926,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00x\x13\x0...
325,06,067,00277298,06067,Sacramento,Sacramento County,06,H1,G4020,472,40900,None,A,2499983887,75425434,+38.4500161,-121.3404408,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00*\x13\x0...
329,06,083,00277306,06083,Santa Barbara,Santa Barbara County,06,H1,G4020,None,42200,None,A,7084063392,2729751706,+34.5370572,-120.0399729,b'\x01\x06\x00\x00\x00\x03\x00\x00\x00\x01\x03...
346,06,009,01675885,06009,Calaveras,Calaveras County,06,H1,G4020,None,None,None,A,2641784992,43841871,+38.1838996,-120.5614415,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x82\x0c...
394,06,111,00277320,06111,Ventura,Ventura County,06,H1,G4020,348,37100,None,A,4771987962,947345370,+34.3587415,-119.1331432,b'\x01\x06\x00\x00\x00\x03\x00\x00\x00\x01\x03...


In [11]:
def get_county_name(latitude, longitude):
    point = gpd.GeoDataFrame(geometry=gpd.points_from_xy([longitude], [latitude]))
    county = gdf_CA[gdf_CA.contains(point.unary_union)]
    return county['NAME'].iloc[0] if not county.empty else None

# Example usage
latitude = 37.7749  # Example latitude
longitude = -122.4194  # Example longitude

county_name = get_county_name(latitude, longitude)
print("County Name:", county_name)


County Name: San Francisco


Exploring NorWeST_Stream_Temperatures_2040s_(Feature_Layer) downloaded as csv file from https://data-usfs.hub.arcgis.com/datasets/0cd39b50f7bb4539aac2a472bc93afe0_1/explore?location=39.832923%2C-112.988051%2C5.95

In [6]:
stream = pd.read_csv("NorWeST_Stream_Temperatures_2040s_Feature_Layer.csv")

In [7]:
stream.head()

,OBJECTID,OBSPRED_ID,S1_93_11,S30_2040D,S32_2080D,COMID,GNIS_NAME,NHDV,SHAPELEN
0,49243889,100189,21.30,21.65,21.91,15934665,,2,0.010215
1,49243890,100190,21.41,21.76,22.03,15934665,,2,0.010162
2,49243891,100191,21.50,21.85,22.11,15934665,,2,0.010135
3,49243892,100192,21.58,21.93,22.19,15934665,,2,0.009902
4,49243893,100193,21.62,21.98,22.24,15934665,,2,0.009975


In [12]:
burn = pd.read_csv("Burn_Severity_Trends.csv")

In [13]:
burn.head()

,X,Y,OBJECTID,FIRE_ID,FIRE_NAME,ASMNT_TYPE,FIRE_TYPE,NODATA_THRESHOLD,GREENNESS_THRESHOLD,LOW_THRESHOLD,...,ACRES,IRWINID,MAP_ID,MAP_PROG,PERIM_ID,DNBR_OFFST,DNBR_STDDV,COMMENTS,PRE_ID,POST_ID
0,-144.673246,64.482309,830721,AK6448214467319840607,GLACIER CREEK,Initial (SS),Wildfire,9999,9999,550,...,1897,NaN,4534,MTBS,506601519860705,-9999,-9999,NaN,NaN,506801519840713
1,-102.849877,43.083926,830722,SD4308410285019840906,SANDOZ,Initial,Wildfire,-970,-150,220,...,2691,NaN,4535,MTBS,NaN,211,-9999,NaN,503303019840622,503303019840926
2,-113.457720,48.804345,830723,MT4880411345819840819,NAPI PEAK,Extended,Wildfire,-970,-150,160,...,3680,NaN,4536,MTBS,NaN,-62,-9999,NaN,504102619840614,504102619850719
3,-106.958289,45.336400,830724,MT4533610695819840826,KIRBY,Extended,Wildfire,-970,-150,150,...,1597,NaN,4537,MTBS,NaN,52,-9999,NaN,503602819840814,503602819850817
4,-106.322839,45.616844,830725,MT4561710632319840918,LAKE CREEK,Initial,Wildfire,-970,-150,-50,...,3617,NaN,4538,MTBS,NaN,-104,-9999,NaN,503602819840814,503602819841017


In [16]:
sylviculture = pd.read_csv("Silviculture_Reforestation_Needs_Feature_Layer.csv")

/var/folders/dd/nb8m5vwd1sz3_s49jh1plcfr0000gn/T/ipykernel_5612/1873578248.py:1: DtypeWarning: Columns (12,32,33,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  sylviculture = pd.read_csv("Silviculture_Reforestation_Needs_Feature_Layer.csv")


In [17]:
sylviculture.head()

,OBJECTID,REGION_CODE,ADMIN_FOREST_CODE,ADMIN_FOREST_NAME,ADMIN_DISTRICT_NAME,DISTRICT_CODE,HOME_ORG,SUID,FACTS_ID,SUBUNIT,...,EDW_INSERT_DATE,REV_DATE,UK_TABULAR,UK,DATA_SOURCE_CODE,REFORESTATION_STATUS_CODE,REFORESTATION_STATUS,SHAPEAREA,SHAPELEN,SHAPE
0,12392704,1,14,KOOTENAI NATIONAL FOREST,LIBBY RANGER DISTRICT,5,11405,011405A120300002000,A120300002,000,...,2021/01/19 14:19:36+00,NaN,5556309010602_011405A120300002000,0114_51222_ACTP,NaN,NaN,NaN,0.000029,0.028016,NaN
1,12392705,2,6,MEDICINE BOW-ROUTT NATIONAL FORESTS,PARKS RANGER DISTRICT,4,20604,020604CAMPCRK111000,CAMPCRK111,000,...,2022/11/22 01:22:17+00,2016/10/28 00:00:00+00,6857758010602_020604CAMPCRK111000,0206_28905_ACTP,3.0,NaN,NaN,0.000008,0.017891,NaN
2,12392706,2,3,BLACK HILLS NATIONAL FOREST,MYSTIC RANGER DISTRICT,6,20306,0203066062410000020,6062410000,020,...,2021/10/07 01:28:50+00,2021/08/19 00:00:00+00,6834159010602_0203066062410000020,0203_167471_ACTP,3.0,NaN,NaN,0.000015,0.023452,NaN
3,12392707,6,4,MALHEUR NATIONAL FOREST,BEAR VALLEY RANGER DISTRICT,1,60401,0604013226CKF220102,3226CKF220,102,...,2023/03/19 01:24:22+00,2023/03/17 00:00:00+00,6731260010602_0604013226CKF220102,0604_245814_ACTP,24.0,NaN,NaN,0.000028,0.027725,NaN
4,12392708,5,5,KLAMATH NATIONAL FOREST,HAPPY CAMP RANGER DISTRICT,52,50552,0505522090017000002,2090017000,002,...,2023/02/10 01:20:57+00,2022/09/29 00:00:00+00,6618471010602_0505522090017000002,0505_35654_ACTP,3.0,NaN,NaN,0.000006,0.011316,NaN
